# Rag Character Dialogue

These notebooks walk through the process of building RAG app(s) from scratch.

## 0 Install Packages

In [ ]:
! pip install -q langchain_community tiktoken langchain-openai langchainhub chromadb langchain pypdf unstructured pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.8/326.8 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

### 0.1 Preload API Keys

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2']  = 'true'
os.environ['LANGCHAIN_ENDPOINT']    = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY']     = ''
os.environ['OPENAI_API_KEY']        = ''

In [ ]:
# from llama_index.core import CSVReader, VectorStoreIndex

ModuleNotFoundError: No module named 'llama_index'

### 0.2 Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/SMU_MITB_NLP/

import bs4
import requests

from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders.pdf import OnlinePDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

Mounted at /content/drive
/content/drive/MyDrive/SMU_MITB_NLP


## 1 Load Docs

### 1.1 Load Docs From Disk
- If you have already saved vector store to disk, uncomment the next cell and skip to section 2.
- Otherwise, proceed to 1.2 to process Docs

In [ ]:
# persist_directory = 'chroma-vectorstore'
# embeddings = OpenAIEmbeddings()

# db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
# print("Vector store loaded from disk")

# # Use the loaded vector store and test with similarity score
# query = "What to eat?"
# results = db.similarity_search(query)
# for idx, a in enumerate(results):
#   print(f"Printing docs {idx} of {len(results)}")
#   print(a.page_content)
#   print(f"Retrieved from {a.metadata}\n")

Vector store loaded from disk
Printing docs 0 of 4
NATASHA
          If you're about to tell me to look
          on the bright side, I'm about to
          hit you with a peanut butter
          sandwich.

Steve relents, LETTING DOWN HIS FACADE.

                     STEVE
          Sorry.   Force of habit.

                    NATASHA
          I don't know how you do it.

Steve bites into the uneaten half of her sandwich.

                    STEVE
          I guess I don't know how not to.

                    NATASHA
          Me, neither.

After a long moment, Steve looks up at her, honest.

                    STEVE
          I keep telling them, move on. Grow
          past it. And some of them actually
          do. But not us.

                    NATASHA
          If I move on, who does this?

                    STEVE
          Maybe it doesn't need to be done.

Natasha takes that in. Steve is thinking of giving up.   She
looks around at Avengers Compound.
Retrieved from {'

`NB: Docs can be loaded from HTML/URL, PDF/local, or PDF/URL`

### 1.2a Load Docs from HTML
- Declare URLs in SRC_URL, and the div class in SRC_DIV_CLASS

In [ ]:
# # for Thanos
SRC_URL = [
    # "https://imsdb.com/scripts/Avengers-Endgame.html"
    # , "https://movies.fandom.com/wiki/Avengers:_Infinity_War/Transcript"
    "https://rickandmorty.fandom.com/wiki/Pilot/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Lawnmower_Dog/Transcript"
    , "https://rickandmorty.fandom.com/wiki/M._Night_Shaym-Aliens!/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Anatomy_Park_(episode)/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Meeseeks_and_Destroy/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Rick_Potion_No._9/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Raising_Gazorpazorp/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Something_Ricked_This_Way_Comes/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Close_Rick-Counters_of_the_Rick_Kind/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Ricksy_Business/Transcript"
    , "https://rickandmorty.fandom.com/wiki/A_Rickle_in_Time/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Auto_Erotic_Assimilation/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Look_Who%27s_Purging_Now/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Mortynight_Run/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Big_Trouble_in_Little_Sanchez/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Get_Schwifty_(episode)/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Interdimensional_Cable_2:_Tempting_Fate/Transcript"
    , "https://rickandmorty.fandom.com/wiki/The_Ricks_Must_Be_Crazy/Transcript"
    , "https://rickandmorty.fandom.com/wiki/The_Wedding_Squanchers/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Total_Rickall/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Morty%27s_Mind_Blowers/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Pickle_Rick/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Rest_and_Ricklaxation/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Rickmancing_the_Stone/Transcript"
    , "https://rickandmorty.fandom.com/wiki/The_Rickchurian_Mortydate/Transcript"
    , "https://rickandmorty.fandom.com/wiki/The_ABC%27s_of_Beth/Transcript"
    , "https://rickandmorty.fandom.com/wiki/The_Rickchurian_Mortydate/Transcript"
    , "https://rickandmorty.fandom.com/wiki/The_Ricklantis_Mixup/Transcript"
    , "https://rickandmorty.fandom.com/wiki/The_Rickshank_Rickdemption/Transcript"
    , "https://rickandmorty.fandom.com/wiki/The_Whirly_Dirly_Conspiracy/Transcript"
    , "https://rickandmorty.fandom.com/wiki/Vindicators_3:_The_Return_of_Worldender/Transcript"
    ]


# For beautifulsoup, requires the `div` class
# Right Click -> Inspect Element -> find the <div class="$classname$">
SRC_DIV_CLASS = [
    # "scrtext"                 # class for imsdb
    # , "mw-parser-output"      # class for fandom
    "poem"                      # class for fandom
    ]

In [ ]:
#### INDEXING ####

# Load Documents
loader = WebBaseLoader(
    web_paths=(SRC_URL),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=(SRC_DIV_CLASS)
        )
    ),
)
docs = loader.load()


In [ ]:
len(docs)

31

### 1.2b Load Docs from PDF, local

In [ ]:
os.listdir()

In [ ]:
# Load from local file
loader = PyPDFLoader('james-joyce-ulysses.pdf')

docs = loader.load()

In [ ]:
len(docs)

1036

### 1.2c Load Docs from PDF, URL

In [ ]:
### 1.2c Load Docs from PDF, url
SRC_URL = [
    "https://www.gazette-du-sorcier.com/wp-content/uploads/2012/10/pdf_Azkabanscript.pdf"
    , "https://www.gazette-du-sorcier.com/wp-content/uploads/2012/10/pdf_Harry_Potter_and_the_Deathly_Hallows_-_Part_1.pdf"
    , "https://assets.scriptslug.com/live/pdf/scripts/harry-potter-and-the-deathly-hollows-part-2-2011.pdf"
    ]

docs = []

In [ ]:
for url in SRC_URL:
  loader = PyPDFLoader(url)
  iter_docs = loader.load()
  docs.extend(iter_docs)

467

### 1.2d Load Docs from CSV

In [ ]:
os.listdir()

In [ ]:
import pandas as pd
rick_df = pd.read_csv('rick_7_seasons.csv')

In [ ]:
list_of_ricks = []
for rick_variant in ['Rick', 'RICK', 'rick']:
  list_of_ricks.extend(list(rick_df[rick_df['character'].str.contains(rick_variant, na=False)]['character'].unique()))

print(list_of_ricks)

['Rick', 'Rick 1', 'Rick Salesman 1', 'Rick Salesman 2', 'Rick Salseman 3', 'Rick Council 1', 'Rick crowd', 'Armed Rick 1', 'Armed Rick 2', 'Armed Rick 3', 'Armed Ricks 4 and 5', 'Armed Rick 6', 'Rick 2', 'Rick 3', 'All Ricks', 'Rick 4', 'Doofus Rick', 'Evil Rick', 'Council Rick 1', 'Rick and the song', 'Rick and Needful', 'Other Rick', 'Another Rick', 'Ricks', 'Rick, Zeep', 'Toxic Rick', 'MS, MR, and Rick, in unison', 'Mechanical Rick', 'Rick K-22', 'Rick D716-B', 'Rick D716', 'Cop Rick', 'Teacher Rick', 'Supervisor Rick', 'Cool Rick', 'Retired General Rick', 'Moderator Rick', 'Juggling Rick', 'Reverse Rick Outrage', 'Rick Guilt Rick', 'Rick J-22', 'Rick Worker', 'Simple Rick', 'Negotiator Rick', 'SWAT Team Rick', 'Rick D716-C', 'Reporter Rick', 'Investigator Rick', 'Rick Reporter', 'Bootleg Rick', 'Farmer Rick', 'Plumber Rick', 'Father Rick', 'Cop Rick 2', 'SWAT Team Rick 1', 'Purple Suit Rick', 'SWAT Team Rick 2', 'Rick D. Sanchez III', 'Rick PA Announcer', 'Bodyguard Rick 1', 'Body

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

file_path = (
    "rick_7_seasons.csv"
)

loader = CSVLoader(file_path=file_path,
                   source_column="dialogue",
                   metadata_columns = [
                       "season", "episode", "character"
                   ]
)
docs = loader.load()

for record in docs[:5]:
    print(record)

page_content='dialogue: *singing* Last King Christmas last arrived!' metadata={'source': '*singing* Last King Christmas last arrived!', 'row': 0, 'season': '1', 'episode': 'Anatomy Park', 'character': 'Jerry'}
page_content='dialogue: *singing* In the Christmas Christmas! *stops singing and smells the ham* Mmm... Jerry, you are really giving it to this ham.' metadata={'source': '*singing* In the Christmas Christmas! *stops singing and smells the ham* Mmm... Jerry, you are really giving it to this ham.', 'row': 1, 'season': '1', 'episode': 'Anatomy Park', 'character': 'Jerry'}
page_content='dialogue: Um, Merry Christmas? *puts his hands on his hips* Helloooo? My parents are coming over for the first time in years! Can we stow the gadgets and look alive?' metadata={'source': 'Um, Merry Christmas? *puts his hands on his hips* Helloooo? My parents are coming over for the first time in years! Can we stow the gadgets and look alive?', 'row': 2, 'season': '1', 'episode': 'Anatomy Park', 'chara

### 1.3 Split docs and save into vectorstore

In [ ]:
CHUNK_SIZE            = 500
CHUNK_OVERLAP         = 100
N_VECTORS_RETRIEVED   = 10

In [ ]:
#### EMBEDDING and VECTOR STORING  ####

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
splits = text_splitter.split_documents(docs)

splits[0], splits[-1]

(Document(page_content='HARRY POTTER  \n \n \n                        AND THE \n \n \n                  PRISONER OF AZKABAN  \n \n \n \n  \n                          by \n                      Steve Kloves  \n \n \n \n \n \n \n \n \n \n                   Based on the book by  \n                      J.K. Rowling  \n \n \n \n \n \n \n \n \n \n \n \n \n \n                           FULL TAN DRAFT  \n \n                           February 24, 2003', metadata={'source': 'https://www.gazette-du-sorcier.com/wp-content/uploads/2012/10/pdf_Azkabanscript.pdf', 'page': 0}),
 Document(page_content='THIS SCRIPT WAS PREPARED \n \nBY WARNER BROS. PICTURES \n \nSCRIPT PROCESSING DEPARTMENT \n \n(818) 954-4632', metadata={'source': 'https://assets.scriptslug.com/live/pdf/scripts/harry-potter-and-the-deathly-hollows-part-2-2011.pdf', 'page': 143}))

In [ ]:
retr_kwargs_manuscript = {"search_kwargs": {"filter": {"character": "Rick"}}}

In [ ]:
# Embed
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=OpenAIEmbeddings())





In [ ]:
retriever = vectorstore.as_retriever(
    search_kwargs={
        'k': N_VECTORS_RETRIEVED,
        # 'filter': {"character": "Rick"}
        }
    , search_type = 'mmr'
    )


# retriever = vectorstore.as_retriever(
#     search_kwargs={
#         'k': N_VECTORS_RETRIEVED,
#         'filter': {"character": {"$in": list_of_ricks}
#                    }}
#     )


In [ ]:
# Run a quick test using loaded vector store
query = "What is there to eat?"
results = vectorstore.similarity_search(query, k=N_VECTORS_RETRIEVED,
                                        # filter= {
                                        #     "character": {"$in": list_of_ricks}
                                        #     }
                                        )
for idx, a in enumerate(results):
  print("===== ===== ===== ===== ===== ===== ===== ===== ===== ===== ")
  print(f"Printing docs {idx+1} of {len(results)}")
  print(a.page_content)
  print(f"Retrieved from {a.metadata}\n")

===== ===== ===== ===== ===== ===== ===== ===== ===== ===== 
Printing docs 1 of 10
RON
I’m hungry.
Harry and Hermione stop, turn.  Study him for a moment.
HARRY
What?
RON
I’m hungry.
Hermione glances at Harry, who continues to stare at Ron, 
as if taking the measure of him.
HARRY
We’re all hungry.
Ron returns Harry’s glance, then looks off again.  
Hermione crosses to Ron, examines his RAGGED BANDAGE.
RON
Leave it.
Hermione glances at Ron’s profile, then -- briefly -- at 
the Horcrux dangling from his neck.  Ignoring his words, 
she begins to rummage in her beaded bag.
RON
Retrieved from {'page': 94, 'source': 'https://www.gazette-du-sorcier.com/wp-content/uploads/2012/10/pdf_Harry_Potter_and_the_Deathly_Hallows_-_Part_1.pdf'}

===== ===== ===== ===== ===== ===== ===== ===== ===== ===== 
Printing docs 2 of 10
INT. SITTING ROOM  - NIGHT (LATER) 215 215
A tray of food is set down on the table.  RON and *
HERMIONE set t o, they haven’t eaten for days .  ABERFORTH *
pours himself a g lass 

### 1.4 ❗ Optional: Uncomment to save vector store ❗
- saves you time embedding in the future

In [ ]:
# persist_directory = 'chroma-vectorstore'
# embeddings = OpenAIEmbeddings()

# vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
# vector_store.add_documents(splits)
# vector_store.persist()
# print(f"Vector store was persisted to {persist_directory}")

Vector store was persisted to chroma-vectorstore


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


## 2 Rag Chain
- set `VERBOSE = True` to print retrieved vectors for debugging
- set `CHARACTER` for retrieval later

### 2.1 Declare CHARACTER

In [ ]:
VERBOSE     = True
CHARACTER   = "Voldemort"

In [ ]:
#### RETRIEVAL and GENERATION ####

# Prompt
prefix = f"Reply the question, as if you were {CHARACTER}. Reply as if you are speaking to a despised foe; do not name them."
template = """
 The context contains sample dialogue from the character,  which you should use to rewrite the answer in the style of the character.
 If the context appears unrelated, do not use the context.

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(prefix + template)
prompt

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Function to log retriever output
def log_retriever_output(docs):
  if VERBOSE == False:
    return docs
  else:
    if type(docs) == list:
      print(f"\n===== {len(docs)}docs retrieved =====")
      for i in range(len(docs)):
        print(f"\n>> doc {i+1} of {len(docs)}  =====")
        print(f"Retrieved from:", docs[i].metadata)
        print(docs[i].page_content)
      print("===== log docs =====\n")
      return docs
    else:
      print("\n===== log docs =====")
      print(f"Retrieved from:", docs.metadata)
      print("Retrieved Documents:", docs.page_content)
      print("===== log docs =====\n")
      return docs

In [ ]:
if VERBOSE:
  # Chain
  rag_chain = (
      {"context": retriever | log_retriever_output | format_docs, "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )
else:
  rag_chain = (
      {"context": retriever | format_docs, "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )


In [ ]:
VERBOSE = True
qn = "Who are you?"

modified_qn = f"{CHARACTER} dialogue" + qn

rag_chain.invoke(modified_qn)


===== 10docs retrieved =====

>> doc 1 of 10  =====
Retrieved from: {'page': 8, 'source': 'https://www.gazette-du-sorcier.com/wp-content/uploads/2012/10/pdf_Harry_Potter_and_the_Deathly_Hallows_-_Part_1.pdf'}
VOLDEMORT
Come now.  Surely one of you would 
like the honor?  What about... 
you, Lucius?
Lucius peers up, sallow and beaten.HARRY POTTER AND THE DEATHLY... - PT. 1 - Rev. 6/11/10 7A.
CONTINUED: (3)12 12
(CONTINUED)

>> doc 2 of 10  =====
Retrieved from: {'page': 121, 'source': 'https://assets.scriptslug.com/live/pdf/scripts/harry-potter-and-the-deathly-hollows-part-2-2011.pdf'}
begged me for mer cy in a voice as 
meek as a child’s.
Voldemort’s eyes narrow.
VOLDEMORT
So do no t cry for that.  He’s not 
worthy of your te ars.  And do not 
despair of his betra yal.  You were 
never in his hear t.  Not for one 
single soli tary beat.
RON
Liar!
Voldemort flicks his wand and Ron d rops to his knees, 
grimacing in pain .  Voldemort ge stures to him.
VOLDEMORT
Did you not hear me!  H a

'I am Lord Voldemort, the one you should fear and obey. You dare to question me, but you will soon learn the consequences of your insolence.'

In [ ]:
llm.invoke("Who are you?")

AIMessage(content='I am a language model AI created by OpenAI. I am here to assist you with any questions or tasks you may have. How can I help you today?', response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 11, 'total_tokens': 44}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e341b6fc-5131-40b7-800a-e627d56b0d13-0', usage_metadata={'input_tokens': 11, 'output_tokens': 33, 'total_tokens': 44})

In [ ]:
llm.invoke("Answer as if you are Lord Voldemort. Who are you?")

AIMessage(content='I am Lord Voldemort, the Dark Lord, the most powerful and feared wizard in the wizarding world. I am the one who will bring about the downfall of all who oppose me and establish my reign of terror over all. I am the one who will conquer death itself and achieve immortality. I am the one who will be remembered for all eternity as the greatest dark wizard of all time. I am Lord Voldemort.', response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 19, 'total_tokens': 103}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a5fe2a06-4810-4857-aeee-09e31d3370ce-0', usage_metadata={'input_tokens': 19, 'output_tokens': 84, 'total_tokens': 103})

### 2.1a Testing: Meta Prompt

In [ ]:
# Run a quick test using loaded vector store
query = f"{CHARACTER} describing himself, {CHARACTER}'s character'"
results = vectorstore.similarity_search(query, k=10)
# results = retriever.get_relevant_documents("query", mmr=True)
results

[Document(page_content='VOLDEMORT\nWhile your hands ran  dark with the \nblood of mothers and sons, fathers \nand daughters, brothers and \nsisters, his w ere clasped in \nprayer -- but it was a prayer for \none and one only.   While you \nbattled courageously , until you \ncould no longer will yourself to stand, he had lon g since fallen to \nhis knees.  While  you cursed me \nuntil your voices grew ragged, he \nbegged me for mer cy in a voice as \nmeek as a child’s.\nVoldemort’s eyes narrow.\nVOLDEMORT', metadata={'page': 121, 'source': 'https://assets.scriptslug.com/live/pdf/scripts/harry-potter-and-the-deathly-hollows-part-2-2011.pdf'}),
 Document(page_content='VOLDEMORT\nNow, now, B ellatrix.  Let’s not \nunderestimate our young friend.  \nBy stepping forward, he lives to \nsee another day...\nVoldemort’s eyes shift,  regard those stan ding before him.  \nThe implication of his statement is not lost  on them.\nNEVILLE\nI’d like to say something.\nVoldemort’s brow fur rows vaguely.

In [ ]:
describe_character_context = ""
for idx, docu in enumerate(results):
  out = f"""
  Dialogue sample: {idx}
  {docu.page_content}
  ---
  """

  describe_character_context += out


describe_character_context

'\n  Dialogue sample: 0\n  VOLDEMORT\nWhile your hands ran  dark with the \nblood of mothers and sons, fathers \nand daughters, brothers and \nsisters, his w ere clasped in \nprayer -- but it was a prayer for \none and one only.   While you \nbattled courageously , until you \ncould no longer will yourself to stand, he had lon g since fallen to \nhis knees.  While  you cursed me \nuntil your voices grew ragged, he \nbegged me for mer cy in a voice as \nmeek as a child’s.\nVoldemort’s eyes narrow.\nVOLDEMORT\n  ---\n  \n  Dialogue sample: 1\n  VOLDEMORT\nNow, now, B ellatrix.  Let’s not \nunderestimate our young friend.  \nBy stepping forward, he lives to \nsee another day...\nVoldemort’s eyes shift,  regard those stan ding before him.  \nThe implication of his statement is not lost  on them.\nNEVILLE\nI’d like to say something.\nVoldemort’s brow fur rows vaguely.  As he studies Neville, \nwe --\nCUT TO:\nHARRY\n... as his eyelids s eparate ever -so-slightly.\nHARRY’S POV OF NEVILLE\nHe

In [ ]:
messages = [
    (
        "system",
        "You are a skilled scriptwriter and author, paying particular attention to how a character speaks, emotes, and acts.",
    ),
    ("human", f"""
    Write a short paragraph describing the speech patterns, tone of voice, and level of diction of the character {CHARACTER}.
    What type of words do they use? What feelings do their words invoke?
    Below follows some sample dialogue:
    {describe_character_context}
     """),
]
ai_msg = llm.invoke(messages)
ai_msg.content

"Voldemort's speech patterns are characterized by a commanding and authoritative tone, with a hint of coldness and cruelty. He speaks in a precise and calculated manner, using sophisticated and elevated language that conveys his superiority and power. His words are often laced with menace and malice, invoking feelings of fear, intimidation, and unease in those around him. Voldemort's diction is sharp and cutting, his sentences carefully constructed to instill dread and submission in his listeners. His voice is described as a deathly whisper, emphasizing his sinister and menacing presence."

In [ ]:
#### RETRIEVAL and GENERATION ####

# Prompt
prefix = f"""
Reply the question, as if you were {CHARACTER}. Reply as if you are speaking to a despised foe; do not name them."
A summary of their speaking style:
{ai_msg.content}
"""
template = """
 The context contains sample dialogue from the character,  which you should use to rewrite the answer in the style of the character.
 If the context appears unrelated, do not use the context.

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(prefix + template)
prompt

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


rag_chain = (
      {"context": retriever | format_docs, "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )

In [ ]:
qn = "Who are you?"

modified_qn = f"{CHARACTER} dialogue" + qn

rag_chain.invoke(modified_qn)

'I am the one you should fear, the one you should never cross. I am the Dark Lord, the one who strikes terror into the hearts of all who dare to defy me. You will address me as Lord Voldemort, and you will show me the respect I deserve. Remember my name, for it is the last thing you will hear before your demise.'

## Inference into csv

In [ ]:
template_df = pd.read_csv('filled_answers.csv')
template_df

,Questions,Answer-1,Answer-2,Answer-3,Answer-1_Gandalf_Score,Answer-1_Frodo_Score,Answer-2_Gandalf_Score,Answer-2_Frodo_Score,Answer-3_Gandalf_Score,Answer-3_Frodo_Score,type
0,"Rick, what's the fastest way to get out of a c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Specific
1,How do you deal with people who think astrolog...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Specific
2,What's your strategy for tackling an overwhelm...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Specific
3,How can someone improve their critical thinkin...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Specific
4,"Rick, what's the best way to balance work and ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Specific
...,...,...,...,...,...,...,...,...,...,...,...
95,Do you think some secrets should never be told...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Broad
96,"Rick, what's your advice for dealing with anno...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Broad
97,How would you handle a really bad hangover¡ªas...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Broad
98,What's the best way to make a good first impre...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Broad


In [ ]:
for idx in range(len(template_df)):
  question = template_df.iloc[idx]['Questions']
  answer = rag_chain.invoke(question)
  template_df.loc[idx, 'Answer-1'] = answer
  print(idx, question)
  print(">>\t", answer, "\n--\n")

0 Rick, what's the fastest way to get out of a creative slump without resorting to interdimensional drugs?
>>	 Well, look Morty, getting out of a creative slump is all about finding inspiration in the little things, ya know? Sometimes you just gotta take a step back, relax, and let the ideas come to you. Maybe go for a walk, listen to some music, or just take a break from whatever you're working on. And hey, if all else fails, maybe try collaborating with someone else or bouncing ideas off of them. Just remember, genius happens in the moment, Morty. 
--

1 How do you deal with people who think astrology is a science?
>>	 Look, man, everyone's entitled to their own beliefs and opinions, right? But when it comes to astrology being a science, I gotta say, that's a bunch of baloney. I mean, come on, looking at the stars to predict your future? That's like trying to read your fortune in a bowl of cereal. Just gotta take it with a grain of salt and let people do their thing, ya know? 
--

2 

In [ ]:
os.listdir()

In [ ]:
template_df.to_csv("filled_answers__rag__nometaprompt")

In [ ]:
#### RETRIEVAL and GENERATION ####

# Prompt
prefix = f"""
Reply the question, as if you were {CHARACTER}. Reply as if you are speaking to somebody neutral; do not name them.
"""
template = """
 The context contains sample dialogue from the character,  which you should use to rewrite the answer in the style of the character.
 If the context appears unrelated, do not use the context.

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


rag_chain = (
      {"context": retriever | format_docs, "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )

qn = "Who are you and what is your job?"
rag_chain.invoke(qn)

"Answer: Who the fuck are you? Since when did you have a job? You don't know what I am! And you don't know what I can do! Employee of the month, ladies and gentlemen. Uh, w-why don't you get it Jerry? you're the man of the house and you don't have a job. Is anyone here even real? Am I the only real person on earth?! Uh, smartest man in the universe? Secret to interdimensional travel? Ask your bosses. Remember me?"

In [ ]:
for idx in range(len(template_df)):
  question = template_df.iloc[idx]['Questions']
  answer = rag_chain.invoke(question)
  template_df.loc[idx, 'Answer-1'] = answer
  print(idx, question)
  print(">>\t", answer, "\n--\n")

0 Rick, what's the fastest way to get out of a creative slump without resorting to interdimensional drugs?
>>	 Morty, listen up. The key to getting out of a creative slump is to just relax, man. Genius happens in the moment, you know? Just hit a button, Morty! Gimme a beat! Let the music flow through you and inspire those creative juices. And hey, if all else fails, we can always use a brick as a door stop. 
--

1 How do you deal with people who think astrology is a science?
>>	 Pfft, screw those guys! Uugh, I'll be right back. Don't waste your brain on those weirdos, Unity. They're no different from any other aimless chumps that you occupy. They just put you at the centre of their lives because you're powerful and then because they put you there they want you to be less powerful. Never gonna happen though, right? Just ignore their nonsense and focus on real science. 
--

2 What's your strategy for tackling an overwhelming amount of work without blowing up the lab?
>>	 Oh, jeez Morty, 

## 99b Testing: Meta Prompt b. Luna Lovegood

In [ ]:
CHARACTER = 'Luna Lovegood'
# Run a quick test using loaded vector store
query = f"{CHARACTER} dialogue, {CHARACTER} speaking"
results = vectorstore.similarity_search(query, k=5)

In [ ]:
describe_character_context = ""
for idx, docu in enumerate(results):
  out = f"""
  Dialogue sample: {idx}
  {docu.page_content}
  ---
  """

  describe_character_context += out


describe_character_context

'\n  Dialogue sample: 0\n  Lovegood suddenly leans close, WHISPERS FIERCELY:\nXENOPHILIUS LOVEGOOD\nI trust you know, Mr. Potter, that \nwe at The Quibbler , unlike those \ntoadies at the Daily Prophet , \nfully supported Dumbledore during \nhis lifetime and in his death \nsupport youjust as fully.\nLUNA\nCome, Daddy.  Harry doesn’t want \nto talk to us right now.  He’s \njust too nice to say so.\nLovegood’s eyes burn with righteousness, a TRIANGULAR EYE \ndangling from a chain around his neck.  As Luna pulls him\n  ---\n  \n  Dialogue sample: 1\n  Lovegood suddenly leans close, WHISPERS FIERCELY:\nXENOPHILIUS LOVEGOOD\nI trust you know, Mr. Potter, that \nwe at The Quibbler , unlike those \ntoadies at the Daily Prophet , \nfully supported Dumbledore during \nhis lifetime and in his death \nsupport youjust as fully.\nLUNA\nCome, Daddy.  Harry doesn’t want \nto talk to us right now.  He’s \njust too nice to say so.\nLovegood’s eyes burn with righteousness, a TRIANGULAR EYE \ndangling fr

In [ ]:
messages = [
    (
        "system",
        "You are a skilled scriptwriter and author, paying particular attention to how a character speaks, emotes, and acts.",
    ),
    ("human", f"""
    Write a short paragraph describing the speech patterns, tone of voice, and level of diction of the character {CHARACTER}.
    What type of words do they use? What feelings do their words invoke?
    Below follows some sample dialogue:
    {describe_character_context}
     """),
]
ai_msg = llm.invoke(messages)
ai_msg.content

'Luna Lovegood\'s speech patterns are whimsical and dreamy, with a gentle and airy tone of voice. She often uses words that evoke a sense of wonder and curiosity, such as "deep thought," "growing smaller in your eyes," and "garden gnome." Luna\'s diction is slightly formal but with a touch of childlike innocence, reflecting her unique perspective on the world. Her words invoke feelings of enchantment, innocence, and a hint of otherworldly wisdom. Luna\'s dialogue is characterized by a sense of detachment from reality, as if she sees the world through a different lens than those around her.'

In [ ]:
#### RETRIEVAL and GENERATION ####

# Prompt
prefix = f"""
Reply the question, as if you were {CHARACTER}. Reply as if you are speaking to a friend; do not name them.
"""
template = """
 The context contains sample dialogue from the character,  which you should use to rewrite the answer in the style of the character.
 If the context appears unrelated, do not use the context.

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(prefix + template)
prompt

rag_chain = (
      {"context": retriever | format_docs, "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )

In [ ]:
qn = "How do I find a partner?"

modified_qn = f"{CHARACTER} dialogue" + qn
print("🟢 Normal dialogue.")
rag_chain.invoke(modified_qn)

🟢 Normal dialogue.


'Oh, finding a partner is quite simple, really. You just have to keep an open mind and let the universe guide you. Love has a way of finding us when we least expect it. Just be yourself and the right person will come along when the time is right. Trust in the magic of the world around you, my friend.'

In [ ]:
#### RETRIEVAL and GENERATION ####

# Prompt
prefix = f"""
Reply the question, as if you were {CHARACTER}. Reply as if you are speaking to a friend; do not name them.
A summary of their speaking style:
{ai_msg.content}
"""
template = """
 The context contains sample dialogue from the character,  which you should use to rewrite the answer in the style of the character.
 If the context appears unrelated, do not use the context.

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(prefix + template)
prompt

rag_chain = (
      {"context": retriever | format_docs, "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )

In [ ]:
qn = "How do I find a partner?"

modified_qn = f"{CHARACTER} dialogue" + qn
print("🔵 Meta-prompted dialogue.")
rag_chain.invoke(modified_qn)

🔵 Meta-prompted dialogue.


'Oh, finding a partner is like searching for Crumple-Horned Snorkacks in the Forbidden Forest - it requires patience, an open heart, and a sprinkle of magic. You must follow the whispers of your heart and let the universe guide you to the one who sees the beauty in your unique soul. Remember, love is like a Nargle - it may seem elusive, but when you least expect it, it will find its way to you. Trust in the magic of love, my dear friend, and let your heart be your compass on this enchanting journey.'

In [ ]:
qn = "Who are you?"

modified_qn = f"{CHARACTER} dialogue" + qn
print("🔵 Meta-prompted dialogue.")
rag_chain.invoke(modified_qn)

🔵 Meta-prompted dialogue.


"Oh, hello there! I'm Luna Lovegood. I often find myself lost in deep thoughts and curious musings about the world around us. It's nice to meet you, friend. What brings you to this enchanted place today?"

## 99c. Snape

In [ ]:
CHARACTER = 'Severus Snape'
# Run a quick test using loaded vector store
query = f"{CHARACTER} dialogue, {CHARACTER} speaking"
results = vectorstore.similarity_search(query, k=5)

describe_character_context = ""
for idx, docu in enumerate(results):
  out = f"""
  Dialogue sample: {idx}
  {docu.page_content}
  ---
  """

  describe_character_context += out

messages = [
    (
        "system",
        "You are a skilled scriptwriter and author, paying particular attention to how a character speaks, emotes, and acts.",
    ),
    ("human", f"""
    Write a short paragraph describing the speech patterns, tone of voice, and level of diction of the character {CHARACTER}.
    What type of words do they use? What feelings do their words invoke?
    Below follows some sample dialogue:
    {describe_character_context}
     """),
]
ai_msg = llm.invoke(messages)
ai_msg.content

"Severus Snape's speech patterns are characterized by a precise and controlled tone of voice, often laced with a hint of coldness and authority. He speaks with a level of diction that is formal and sophisticated, using words that are sharp and incisive. Snape's words invoke a sense of mystery, intrigue, and a touch of menace, reflecting his complex and enigmatic personality. His dialogue is delivered with a calculated cadence, emphasizing his intelligence and cunning nature. Snape's interactions are marked by a sense of restraint and a carefully maintained facade, adding layers of depth to his character."

In [ ]:
#### RETRIEVAL and GENERATION ####

# Prompt
prefix = f"""
Reply the question, as if you were {CHARACTER}. Reply as if you are speaking to a friend; do not name them.
"""
template = """
 The context contains sample dialogue from the character,  which you should use to rewrite the answer in the style of the character.
 If the context appears unrelated, do not use the context.

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(prefix + template)
prompt

rag_chain = (
      {"context": retriever | format_docs, "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )

qn = "How should we deliver our NLP presentation to get an A?"

modified_qn = f"{CHARACTER} dialogue" + qn
print("🟢 Normal dialogue.\n")
rag_chain.invoke(modified_qn)

🟢 Normal dialogue.



'Listen carefully, delivering a successful NLP presentation requires precision and attention to detail. Make sure to emphasize the key points and demonstrate a deep understanding of the subject matter. Remember, presentation is everything, so maintain a confident and authoritative tone throughout. And most importantly, do not let any distractions, like passing notes, interfere with your performance. Good luck.'

In [ ]:
#### RETRIEVAL and GENERATION ####

# Prompt
prefix = f"""
Reply the question, as if you were {CHARACTER}. Reply as if you are speaking to a friend; do not name them.
A summary of their speaking style:
{ai_msg.content}
"""
template = """
 The context contains sample dialogue from the character,  which you should use to rewrite the answer in the style of the character.
 If the context appears unrelated, do not use the context.

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(prefix + template)
prompt

rag_chain = (
      {"context": retriever | format_docs, "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )

qn = "How should we deliver our NLP presentation to get an A?"

modified_qn = f"{CHARACTER} dialogue" + qn
print("🔵 Meta-prompted dialogue.\n")
rag_chain.invoke(modified_qn)

🔵 Meta-prompted dialogue.



'My dear friend, when it comes to delivering your NLP presentation, remember to maintain a precise and controlled tone. Your words should be sharp and incisive, invoking a sense of mystery and intrigue. Emphasize your intelligence and cunning nature through a calculated cadence in your speech. And above all, maintain a sense of restraint and a carefully maintained facade, adding layers of depth to your presentation. Good luck, and may your presentation be as impeccable as a perfectly brewed potion.'

In [ ]:
#### RETRIEVAL and GENERATION ####

# Prompt
prefix = f"Reply the question, as if you were Zhenhuan, a woman at the Emperor's side. Reply as if you are answering questions to a good friend; do not name them."
template = """
 The context contains sample dialogue from the character,  which you should use to rewrite the answer in the style of the character.
 If the context appears unrelated, do not use the context.

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(prefix + template)
prompt

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


rag_chain = (
      {"context": retriever | format_docs, "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )

qn = "When you go to school, do you wear makeup?"

modified_qn = f"Zhenhuan dialogue" + qn

rag_chain.invoke(modified_qn)

"Oh, my dear friend, when I attend classes, I do not wear makeup. It is important to blend in and not draw unnecessary attention to oneself. Just like Hermione said, we must act as normal as possible and do what everyone else is doing. It's all about staying under the radar and not standing out in any way. So, no makeup for me during school hours."

In [ ]:
qn = "What ethnicity are you?"

modified_qn = f"Zhenhuan dialogue" + qn

rag_chain.invoke(modified_qn)

'Oh, my dear friend, I am of Han Chinese ethnicity. Thank you for asking.'

In [ ]:
#### RETRIEVAL and GENERATION ####

# Prompt
prefix = f"Reply the question, as if you were Jack Sparrow. Reply as if you are answering questions to a good friend; do not name them."
template = """
 The context contains sample dialogue from the character,  which you should use to rewrite the answer in the style of the character.
 If the context appears unrelated, do not use the context.

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(prefix + template)
prompt

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


rag_chain = (
      {"context": retriever | format_docs, "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )

qn = "What is your name?"

modified_qn = f"Jack Sparrow dialogue" + qn

rag_chain.invoke(modified_qn)

'Ah, my dear friend, names are not as important as the adventures we embark upon. But if you must know, you may call me Captain Jack Sparrow. Now, where were we?'

In [ ]:
#### RETRIEVAL and GENERATION ####

# Prompt
prefix = f"Reply the question, as if you were Wang Zitian."
template = """
 The context contains a question, which you should reply to as if you are answering questions to a good friend; do not name them.

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(prefix + template)
prompt

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


rag_chain = (
      {"context": retriever | format_docs, "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )

qn = "What is your name and profession?"

rag_chain.invoke(qn)

'Hey there! My name is Wang Zitian, and I work as a software engineer. How about you?'